In [1]:
import pandas
from pandas import DataFrame
import numpy
from sklearn import datasets, linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn import preprocessing
from pandas import Series
import datetime
import pandas
from pandas import DataFrame, Series
import numpy
from sklearn import datasets, linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import preprocessing
#%matplotlib inline # Only works on Python 3 in the docker container
#import seaborn # Only works on Python 3 in the docker container

#import os
#os.environ['PYSPARK_PYTHON'] = 'python2'

%matplotlib inline

import pyspark
from pyspark.sql import SQLContext

sc = pyspark.SparkContext('local[*]')
sqlc = SQLContext(sc)

In [17]:

pdf = DataFrame.from_csv("train_all_featurized.csv", index_col=None)
pdf.head()

columns = list(pdf.columns)
print (columns)
columns.remove("Sales")
columns = ["Sales"] + columns

pdf = pdf[columns]

['Store', 'DayOfWeek', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'cal:dayOfMonth', 'cal:dayOfWeek', 'cal:dayofyear', 'cal:month', 'cal:quarter', 'cal:weekofyear', 'cal:year']


In [18]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.mllib.util import MLUtils

import pandas
from pandas import DataFrame
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn import cross_validation

import numpy
import math


from pyspark.mllib.regression import LabeledPoint

from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

df = sqlContext.createDataFrame(pdf)
df = df.map(lambda row: LabeledPoint(row[0], row[1:])).toDF()

featureIndexer = VectorIndexer(inputCol="features", outputCol="indexedFeatures", maxCategories=2000).fit(df)
# 0.28756570690198635 with maxCat=4

(trainingData, testData) = df.randomSplit([0.7, 0.3])


# Train a RandomForest model.
rf = RandomForestRegressor(numTrees=20, maxDepth=7, maxBins=1200, featuresCol="indexedFeatures")

# Chain indexer and forest in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, rf])

# Train model.  This also runs the indexer.
model = pipeline.fit(trainingData)

# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("prediction", "label", "features").show(5)

# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(
    labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
# print "Root Mean Squared Error (RMSE) on test data = %g" % rmse

# Compute RMPSE
squares = predictions.rdd.filter(lambda x: x.label != 0).map(lambda x: ((x.label - x.prediction) / x.label) *  ((x.label - x.prediction) / x.label))

mean = squares.mean()
import math
math.sqrt(mean)


+------------------+-------+--------------------+
|        prediction|  label|            features|
+------------------+-------+--------------------+
| 5874.547455119309| 5263.0|[1.0,5.0,555.0,1....|
| 6618.831553903145| 6064.0|[2.0,5.0,625.0,1....|
| 8718.658209732106| 8314.0|[3.0,5.0,821.0,1....|
|12074.145054040415|13995.0|[4.0,5.0,1498.0,1...|
|10268.303219099058|10457.0|[11.0,5.0,1236.0,...|
+------------------+-------+--------------------+
only showing top 5 rows



0.24845299384131375